<a href="https://colab.research.google.com/github/rfasich24/Skripsi-E-Nose/blob/main/Memperbaiki_Dataset_Mentah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import glob
import os

csv_files = glob.glob("*.csv")

for file_path in csv_files:
    with open(file_path, 'r') as file:
        file_content = file.read()
    if 'Counter' not in df.columns:
        # Re-read the CSV, potentially with no header initially to inspect the first row
        df = pd.read_csv(file_path, sep=',', header=None)
        # Assuming the first row contains the header, assign it and re-read with the correct header
        new_header = df.iloc[0]
        df = df[1:].copy()
        df.columns = new_header.str.strip() # Strip whitespace from column names
        # Reset index after removing the header row
        df.reset_index(drop=True, inplace=True)
    # Replace semicolon with comma
    modified_content = file_content.replace(';', ',')

    with open(file_path, 'w') as file:
        file.write(modified_content)

print("Separator changed from ';' to ',' for all CSV files.")

Separator changed from ';' to ',' for all CSV files.


# Task
Baca semua file csv yang ada pada runtime. lalu periksa setiap baris termasuk nilai di kolom Counter, jika terdapat baris selain header yang nilainya NaN atau karakter tidak jelas, atau nilai yang aneh hapus baris tersebut lalu gantikan nilainya dengan nilai baris sebelumnya dengan counter yang tetap melanjutkan counter sebelumnya. terakhir pastikan seluruh nilai tipenya adalah numerik dimana kolom (MQ2, MQ3, MQ4, MQ7, MQ8, MQ135, MQ137, TGS) tipe datanya integer, dan kolom (Suhu_C, Kelembaban) tipe data Float dengan maksimal 2 angka dibelakang koma. Timpa file asli lalu download semua file dalam bentuk zip

## Dapatkan daftar file csv

### Subtask:
Identifikasi semua file CSV yang ada di lingkungan runtime.


**Reasoning**:
Use the `glob` module to find all CSV files in the current directory and print the list.



In [9]:
csv_files = glob.glob("*.csv")
print(csv_files)

['Sample 3 Grade 1.csv', 'Sample 1 Grade 1.csv', 'Sample 4 Grade 1.csv', 'Sample 5 Grade 1.csv', 'Sample 2 Grade 1.csv']


## Proses setiap file csv

### Subtask:
Baca file CSV, tentukan separator yang ada saat ini. Ganti separator yang ada dengan koma. Timpa file asli dengan konten yang sudah diubah separatornya.


**Reasoning**:
Process each CSV file to handle missing/invalid values, correct the counter, and enforce specific data types for columns before saving the modified file.



In [12]:
import pandas as pd
import numpy as np

for file_path in csv_files:
    # Read the file with the correct separator (which is now a comma) and ensure header is correctly parsed
    df = pd.read_csv(file_path, sep=',')

    # Check if 'Counter' column exists, if not, try to infer the correct header
    if 'Counter' not in df.columns:
        # Re-read the CSV, potentially with no header initially to inspect the first row
        df = pd.read_csv(file_path, sep=',', header=None)
        # Assuming the first row contains the header, assign it and re-read with the correct header
        new_header = df.iloc[0]
        df = df[1:].copy()
        df.columns = new_header.str.strip() # Strip whitespace from column names
        # Reset index after removing the header row
        df.reset_index(drop=True, inplace=True)

    # Now that we are sure 'Counter' column exists, proceed with processing
    # Ensure 'Counter' is treated as numeric before checking for NaN or invalid values
    df['Counter'] = pd.to_numeric(df['Counter'], errors='coerce')

    # Handle problematic rows and correct the counter
    for i in range(1, len(df)):
        # Check for any NaN in the row or if Counter is NaN or not incrementing correctly
        if df.iloc[i].isnull().any() or pd.isna(df.iloc[i]['Counter']) or (i > 0 and df.iloc[i]['Counter'] != df.iloc[i-1]['Counter'] + 1):
             df.iloc[i] = df.iloc[i-1].copy() # Use .copy() to avoid SettingWithCopyWarning
             df.loc[i, 'Counter'] = df.loc[i-1, 'Counter'] + 1 # Ensure counter increments

    # Convert data types and handle errors by filling with previous valid entry
    int_cols = ['MQ2', 'MQ3', 'MQ4', 'MQ7', 'MQ8', 'MQ135', 'MQ137', 'TGS']
    float_cols = ['Suhu_C', 'Kelembaban']

    for col in int_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64') # Use Int64 to allow NaN
        df[col] = df[col].ffill() # Use ffill()
        df[col] = df[col].astype(int) # Convert to native int after filling


    for col in float_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].ffill() # Use ffill()
        df[col] = df[col].round(2)

    # Convert 'Counter' to integer type before saving
    df['Counter'] = df['Counter'].astype(int)

    # Save the modified DataFrame back to the original file
    df.to_csv(file_path, index=False, sep=',')

print("CSV files processed and saved.")

CSV files processed and saved.


## Buat file zip

### Subtask:
Kompres semua file CSV yang dimodifikasi menjadi satu file zip.


**Reasoning**:
Compress all modified CSV files into a single zip file.



In [13]:
import zipfile
import os
from google.colab import files

zip_file_name = 'Modified_Dataset.zip'

with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for file in csv_files:
        if os.path.exists(file):
            zipf.write(file, os.path.basename(file))
        else:
            print(f"Warning: File {file} not found, skipping.")

print(f"Successfully created {zip_file_name} containing the modified CSV files.")

files.download('Modified_Dataset.zip')

Successfully created Grade_1.zip containing the modified CSV files.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>